# twitterapiv2

This library offers access through Scala to the twitter API v2 endpoints. Additional aids for pagination and throttle management are also provided.

### Set up

You have a number of options to add the lib and dependencies to the classpath:
1. Uber-jar
2. Classes (Windows path)
3. Classes (Windows/WSL path)
4. Classes (Docker path

To generate the uber-jar for the first option simply run the command `assembly` in the sbt prompt. To update the full class path files run the command `Compile/fullClasspath/exportToAmmoniteScript`in the sbt prompt. 

In [8]:
// import $cp.target.`scala-2.13`.`twitterv2-0.1.jar`
import $file.`fullClasspath-Compile-WIN`

import $file.$                          

### Imports and dependencies

In [9]:
import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._
import akka.actor.typed.ActorSystem
import akka.actor.typed.scaladsl.Behaviors
import caseapp._
import dev.habla.twitter.v2.recents._
import scala.util.Success
import scala.util.Failure
import dev.habla.twitter.v2._

import scala.concurrent.{Future, Await, ExecutionContext, duration}, duration._

import akka.actor.typed.ActorSystem

import akka.actor.typed.scaladsl.Behaviors

import caseapp._

import dev.habla.twitter.v2.recents._

import scala.util.Success

import scala.util.Failure

import dev.habla.twitter.v2._

Common dependencies for actor-based systems and Akka stream:

In [10]:
implicit val system = ActorSystem(Behaviors.empty, "TwitterV2")
implicit val ec = system.executionContext

11:59:42.303 [TwitterV2-akka.actor.default-dispatcher-3] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started


system: ActorSystem[Any] = akka://TwitterV2
ec: concurrent.ExecutionContextExecutor = Dispatcher[akka.actor.default-dispatcher]

Obtain the bearer token from the environment. Remember to pass this variable to docker, in case you started this notebook from there (`$ docker run -e <var_name> ...`)

In [4]:
def bearerToken = scala.util.Properties.envOrElse("BEARER_TOKEN", "undefined")

defined function bearerToken

### Search recent endpoint: single requests

We can access programmatically the [search/recent](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) endpoint to obtain the response of a single request as follows:

In [5]:
val response: Future[recents.SingleResponse] = 
    recents.single.Run(recents.SingleRequest("scala3", bearerToken, max_results=Some(10)))

response: Future[SingleResponse] = Success(
  ErroneousJsonSingleResponse(
    JsObject(
      TreeMap(
        "detail" -> JsString("Unauthorized"),
        "status" -> JsNumber(401),
        "title" -> JsString("Unauthorized"),
        "type" -> JsString("about:blank")
      )
    )
  )
)

In [8]:
show(Await.result(response, 1.second))

Tweets(
  Body(
    Some(
      List(
        JsObject(
          TreeMap(
            "author_id" -> JsString("1392851665695424514"),
            "conversation_id" -> JsString("1393971775915106305"),
            "created_at" -> JsString("2021-05-16T16:49:37.000Z"),
            "entities" -> JsObject(
              TreeMap(
                "mentions" -> JsArray(
                  Vector(
                    JsObject(
                      TreeMap(
                        "end" -> JsNumber(15),
                        "start" -> JsNumber(0),
                        "username" -> JsString("sabinehonfleur")
                      )
                    )
                  )
                )
              )
            ),
            "id" -> JsString("1393971913496616964"),
            "in_reply_to_user_id" -> JsString("1214181675501940736"),
            "lang" -> JsString("it"),
            "possibly_sensitive" -> false,
            "public_metrics" -> JsObject(
              TreeMap(
    

            "conversation_id" -> JsString("1393963836806418432"),
            "created_at" -> JsString("2021-05-16T16:46:45.000Z"),
            "entities" -> JsObject(
              TreeMap(
                "mentions" -> JsArray(
                  Vector(
                    JsObject(
                      TreeMap(
                        "end" -> JsNumber(8),
                        "start" -> JsNumber(0),
                        "username" -> JsString("magdben")
                      )
                    )
                  )
                )
              )
            ),
            "id" -> JsString("1393971191707312130"),
            "in_reply_to_user_id" -> JsString("22317128"),
            "lang" -> JsString("fr"),
            "possibly_sensitive" -> false,
            "public_metrics" -> JsObject(
              TreeMap(
                "like_count" -> JsNumber(0),
                "quote_count" -> JsNumber(0),
                "reply_count" -> JsNumber(0),
                "retw

                        "start" -> JsNumber(51),
                        "tag" -> JsString("SCOOBYDOOCMS")
                      )
                    )
                  )
                ),
                "mentions" -> JsArray(
                  Vector(
                    JsObject(
                      TreeMap(
                        "end" -> JsNumber(15),
                        "start" -> JsNumber(3),
                        "username" -> JsString("leonhartcms")
                      )
                    )
                  )
                ),
                "urls" -> JsArray(
                  Vector(
                    JsObject(
                      TreeMap(
                        "display_url" -> JsString("pic.twitter.com/0BfOsZJ7p8"),
                        "end" -> JsNumber(88),
                        "expanded_url" -> JsString(
                          "https://twitter.com/leonhartcms/status/1393663911220633604/photo/1"
                        ),
                

### Search recent endpoint: pagination

But the library can also do pagination and throttle management for us automatically:

In [12]:
val response: Future[recents.PaginatedResponse] = 
    recents.pagination.Run(recents.Pagination(recents.SingleRequest("scala3 -is:retweet", bearerToken), "output", Some(40)))

response: Future[PaginatedResponse] = Success(
  Ok(
    Some(
      Meta(
        Some("1393971876871823361"),
        Some("1393589624853786629"),
        40,
        Some("b26v89c19zqg8o3foswrtt2sadi8b8jjex5vjg7lgh8jh")
      )
    )
  )
)